# Visualisierung von Geodaten, Netzwerken und 3d-Plots

Eine Zusammenstellung von verschiedenen Visualisierungen, die ohne zusätzlicher Konfiguration mit Jupyter Notebook möglich sind.
Bei der Auswahl kommen viele Anwendungsfälle zu kurz und es gibt viele tolle Alternativen zu den vorgestellten Bibliotheken!

## Geodaten

In [ ]:
import pandas as pd
import json
import requests
import folium
import random

Folgendes Beispiel stammt von https://python-visualization.github.io/folium/quickstart.html - vielen Dank an die Autoren!

In [ ]:
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json"
state_unemployment = f"{url}/US_Unemployment_Oct2012.csv"
state_data = pd.read_csv(state_unemployment)

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=state_data,
    columns=["State", "Unemployment"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Unemployment Rate (%)",
).add_to(m)

folium.LayerControl().add_to(m)

m

Folgendes Beispiel stammt von https://github.com/python-visualization/folium/blob/master/examples/HeatMapWithTime.ipynb - vielen Dank an die Autoren!

In [ ]:
import folium
import folium.plugins as plugins
import numpy as np

np.random.seed(3141592)
initial_data = np.random.normal(size=(100, 2)) * np.array([[1, 1]]) + np.array(
    [[48, 5]]
)

move_data = np.random.normal(size=(100, 2)) * 0.01

data = [(initial_data + move_data * i).tolist() for i in range(100)]

weight = 1  # default value
for time_entry in data:
    for row in time_entry:
        row.append(weight)

m = folium.Map([48.0, 5.0], tiles="stamentoner", zoom_start=6)
hm = plugins.HeatMapWithTime(data)
hm.add_to(m)

m

Und eine Fortsetzung des vorherigen Beispiels:

In [ ]:
from datetime import datetime, timedelta

time_index = [
    (datetime.now() + k * timedelta(1)).strftime("%Y-%m-%d") for k in range(len(data))
]


m = folium.Map([48.0, 5.0], tiles="stamentoner", zoom_start=6)

hm = plugins.HeatMapWithTime(data, index=time_index, auto_play=True, max_opacity=0.3)

hm.add_to(m)

m

## Netzwerke

Netzwerke können als Graphen mit Knoten und Kanten repräsentiert werden.
Diese können auch in Python erstellt und im Jupyter Notebook dann visualisiert werden.

Installiere die benötigte Bibliothek on-the-fly

In [ ]:
import networkx as nx
import ipycytoscape

Folgendes Beispiel stammt von https://github.com/QuantStack/ipycytoscape/tree/master/examples - vielen Dank an die Autoren!

In [ ]:
G = nx.complete_graph(4)
cyto = ipycytoscape .CytoscapeWidget()
cyto.graph.add_graph_from_networkx(G)
cyto

Und eine Fortsetzung des vorherigen Beispiels.

In [ ]:
G = nx.complete_graph(6)
for s, t, data in G.edges(data=True):
    if random.random() > .5:
        G[s][t]['classes'] = 'directed'

mixed = ipycytoscape.CytoscapeWidget()
mixed.graph.add_graph_from_networkx(G)
mixed

## 3d-Plots

Folgendes Beispiel stammt von https://matplotlib.org/3.1.1/gallery/mplot3d/surface3d.html - vielen Dank an die Autoren!

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

fig = plt.figure(figsize=[10, 10])
ax = fig.gca(projection='3d')

# Make data.
X = np.arange(-5, 5, 0.25)
Y = np.arange(-5, 5, 0.25)
X, Y = np.meshgrid(X, Y)
R = np.sqrt(X**2 + Y**2)
Z = np.sin(R)

# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
ax.set_zlim(-1.01, 1.01)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
fig = plt.figure(figsize=[10, 10])
ax = fig.gca(projection='3d')

# Data for a three-dimensional line
zline = np.linspace(0, 15, 1000)
xline = np.sin(zline)
yline = np.cos(zline)
ax.plot3D(xline, yline, zline, 'gray')

# Data for three-dimensional scattered points
zdata = 15 * np.random.random(100)
xdata = np.sin(zdata) + 0.1 * np.random.randn(100)
ydata = np.cos(zdata) + 0.1 * np.random.randn(100)
ax.scatter3D(xdata, ydata, zdata, c=zdata, cmap='Greens')
plt.show()

## Zusatz: Benutzeroberfläche in Jupyter Notebook integrieren

In [ ]:
from ipywidgets import ColorPicker, link, HBox
from ipycanvas import RoughCanvas, hold_canvas

width = 1000
height = 1000

Das folgende Beispiel stammt von https://github.com/martinRenou/ipycanvas/blob/master/examples/ - vielen Dank an die Autoren!

In [ ]:
canvas = RoughCanvas(width=width, height=height)

drawing = False
position = None
shape = []


def on_mouse_down(x, y):
    global drawing
    global position
    global shape

    drawing = True
    position = (x, y)
    shape = [position]


def on_mouse_move(x, y):
    global drawing
    global position
    global shape

    if not drawing:
        return

    with hold_canvas(canvas):
        canvas.stroke_line(position[0], position[1], x, y)

        position = (x, y)

    shape.append(position)


def on_mouse_up(x, y):
    global drawing
    global position
    global shape

    drawing = False

    with hold_canvas(canvas):
        canvas.stroke_line(position[0], position[1], x, y)
        canvas.fill_polygon(shape)

    shape = []


canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)

canvas.stroke_style = '#749cb8'

picker = ColorPicker(description='Color:', value='#749cb8')

link((picker, 'value'), (canvas, 'stroke_style'))
link((picker, 'value'), (canvas, 'fill_style'))

HBox((canvas, picker))